# Parsing HTML, Beautiful Soup, Mongo DB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

- Наименование вакансии.
- Предлагаемую зарплату (отдельно минимальную и максимальную).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [43]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [44]:
def _parser_item_hh(item):
    vacancy_date = {}
    # vacancy_name
    vacancy_name = item.find('span', {'class': 'resume-search-item__name'}) \
                    .getText() \
                    .replace(u'\xa0', u' ')   
    vacancy_date['vacancy_name'] = vacancy_name    
    # company_name
    company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info'}) \
                    .find('a') \
                    .getText() 
    vacancy_date['company_name'] = company_name
    # city
    city = item.find('span', {'class': 'vacancy-serp-item__meta-info'}) \
                .getText() \
                .split(', ')[0]
    vacancy_date['city'] = city
    #metro station
    metro_station = item.find('span', {'class': 'vacancy-serp-item__meta-info'}).findChild()
    if not metro_station:
        metro_station = None
    else:
        metro_station = metro_station.getText()
    vacancy_date['metro_station'] = metro_station
    #salary
    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if not salary:
        salary = None
    else:
        salary = salary.getText()
    vacancy_date['salary'] = salary
    # link
    vacancy_link = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
    vacancy_date['vacancy_link'] = vacancy_link 
    # site
    vacancy_date['site'] = 'hh.ru'
    return vacancy_date

In [45]:
def _parser_hh(vacancy):

    vacancy_date = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }

    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'bloko-button'})[-2].getText())
            
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_date.append(_parser_item_hh(item))
                
    return vacancy_date

In [46]:
def parser_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(_parser_hh(vacancy))
    
    df = pd.DataFrame(vacancy_date)

    return df

In [47]:
vacancy = 'Python'
df = parser_vacancy(vacancy)

In [48]:
df

,vacancy_name,company_name,city,metro_station,salary,vacancy_link,site
0,Backend разработчик на Python,Gigsty,Санкт-Петербург,None,250 000 – 300 000 руб.,https://hh.ru/vacancy/47103960?from=vacancy_se...,hh.ru
1,Senior Python developer,КА Натальи Зотовой,Москва,None,до 5 500 USD,https://hh.ru/vacancy/47139730?from=vacancy_se...,hh.ru
2,Python-разработчик,Сбер. IT,Москва,None,None,https://hh.ru/vacancy/47084105?from=vacancy_se...,hh.ru
3,Math + Python developer,ООО Чисельні Технології,Киев,None,None,https://hh.ru/vacancy/47174921?from=vacancy_se...,hh.ru
4,Backend разработчик (Python),ИП CyberTech.kz,Алматы,Абая,300 000 – 600 000 KZT,https://hh.ru/vacancy/47039260?from=vacancy_se...,hh.ru
...,...,...,...,...,...,...,...
395,Backend разработчик на Python / Java в Яндекс....,Яндекс,Москва,Парк культуры,None,https://hh.ru/vacancy/46606188?from=vacancy_se...,hh.ru
396,Middle Python QA Engineer,Национальный исследовательский университет Выс...,Москва,None,100 000 – 100 000 руб.,https://hh.ru/vacancy/46582713?from=vacancy_se...,hh.ru
397,Преподаватель Программирования Python (Yandex ...,МАОУ ОЦ Горностай,Новосибирск,None,None,https://hh.ru/vacancy/44275861?from=vacancy_se...,hh.ru
398,Middle Web Developer (Python/ExtJs),"МАГНИТ, Розничная сеть",Санкт-Петербург,None,None,https://hh.ru/vacancy/46608914?from=vacancy_se...,hh.ru
